In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
Path='/content/drive/MyDrive/Project'

In [ ]:
import tensorflow as tf

IMAGE_SIZE = (256, 256)
BATCH_SIZE = 32

train_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    Path,
    image_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    shuffle=True,
    seed=123,
    validation_split=0.2,
    subset='training'
)

validation_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    Path,
    image_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    shuffle=True,
    seed=123,
    validation_split=0.2,
    subset='validation'
)


Found 2192 files belonging to 3 classes.
Using 1754 files for training.
Found 2192 files belonging to 3 classes.
Using 438 files for validation.


In [ ]:
from tensorflow.keras import layers, models

model = models.Sequential([
    layers.Rescaling(1./255, input_shape=(256, 256, 3)),
    layers.Conv2D(32, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(len(train_dataset.class_names), activation='softmax')
])


/usr/local/lib/python3.11/dist-packages/keras/src/layers/preprocessing/tf_data_layer.py:19: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [ ]:
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)


In [ ]:
history = model.fit(
    train_dataset,
    validation_data=validation_dataset,
    epochs=10
)


Epoch 1/10
55/55 ━━━━━━━━━━━━━━━━━━━━ 372s 7s/step - accuracy: 0.6197 - loss: 1.7077 - val_accuracy: 0.8813 - val_loss: 0.3291
Epoch 2/10
55/55 ━━━━━━━━━━━━━━━━━━━━ 307s 5s/step - accuracy: 0.8791 - loss: 0.3326 - val_accuracy: 0.8995 - val_loss: 0.2858
Epoch 3/10
55/55 ━━━━━━━━━━━━━━━━━━━━ 315s 5s/step - accuracy: 0.9259 - loss: 0.1863 - val_accuracy: 0.9110 - val_loss: 0.3198
Epoch 4/10
55/55 ━━━━━━━━━━━━━━━━━━━━ 327s 5s/step - accuracy: 0.9526 - loss: 0.1141 - val_accuracy: 0.9452 - val_loss: 0.1923
Epoch 5/10
55/55 ━━━━━━━━━━━━━━━━━━━━ 333s 6s/step - accuracy: 0.9815 - loss: 0.0605 - val_accuracy: 0.9406 - val_loss: 0.2558
Epoch 6/10
55/55 ━━━━━━━━━━━━━━━━━━━━ 284s 5s/step - accuracy: 0.9561 - loss: 0.1010 - val_accuracy: 0.8790 - val_loss: 0.3241
Epoch 7/10
55/55 ━━━━━━━━━━━━━━━━━━━━ 342s 6s/step - accuracy: 0.9394 - loss: 0.1563 - val_accuracy: 0.9406 - val_loss: 0.2203
Epoch 8/10
55/55 ━━━━━━━━━━━━━━━━━━━━ 302s 5s/step - accuracy: 0.9773 - loss: 0.0565 - val_accuracy: 0.8402 - v

In [ ]:
# Save the entire model as a .keras file
model.save('classify_leaf.keras')

In [ ]:
loss, accuracy = model.evaluate(validation_dataset)
print(f'Validation accuracy: {accuracy:.2f}')

14/14 ━━━━━━━━━━━━━━━━━━━━ 23s 2s/step - accuracy: 0.9173 - loss: 0.2551
Validation accuracy: 0.92


## Prediction

In [ ]:
import numpy as np
from tensorflow.keras.preprocessing import image

new_model = tf.keras.models.load_model('classify_leaf.keras')

img_path = '/content/drive/MyDrive/Project/Potato___Late_blight/0eb24a67-a174-43db-86c7-cca8795942a2___RS_LB 4722.JPG'
img = image.load_img(img_path, target_size=IMAGE_SIZE)
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)

predictions = new_model.predict(img_array)
predicted_class = train_dataset.class_names[np.argmax(predictions)]

print(f'The image is classified as: {predicted_class}')

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 169ms/step
The image is classified as: Potato___Late_blight
